# AI-A-2023 PJ1-part2

In [2]:
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

## load dataset

In [3]:
train_tfm = transforms.Compose([
    transforms.RandomAffine(0, (0.1, 0.1)),
    transforms.RandomRotation(8),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

test_tfm = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

class ChineseDataset(Dataset):
    def __init__(self, path, tfm, files=None):
        super(ChineseDataset).__init__()
        self.path = path
        self.files = [os.path.join(path, x, y) for x in os.listdir(path)
                      for y in os.listdir(os.path.join(path, x))]
        if files != None:
            self.files = files
        self.transform = tfm

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        idx = int(fname.split("/")[-2])

        return im, idx - 1

## network

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 20, 5),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),

            nn.Conv2d(20, 50, 5),
            nn.BatchNorm2d(50),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
        )
        self.fullyConnected = nn.Sequential(
            nn.Linear(50*4*4, 500),
            nn.ReLU(),
            nn.Linear(500, 12)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fullyConnected(out)

## hyper parameters

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CNN().to(device)
batch_size = 16
epochs = 20
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

split the dataset

In [6]:
data_set = ChineseDataset('./train_data/train', tfm=train_tfm)
train_size = int(len(data_set) * 0.7)
valid_size = int(len(data_set) * 0.2)
test_size = len(data_set) - valid_size - train_size
train_set, valid_set, test_set = torch.utils.data.random_split(
    data_set, [train_size, valid_size, test_size])

train_loader = DataLoader(train_set, batch_size=batch_size,
                          shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size,
                          shuffle=True, pin_memory=True)

## training

In [12]:
from torch.utils.tensorboard import SummaryWriter

In [13]:
best_acc = 0
writer = SummaryWriter()
for epoch in range(epochs):
    # ------------训练部分-----------
    model.train()
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.long().to(device)
        
        logits = model(imgs)
        loss = loss_function(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        # grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()
        # 计算loss与准确率
        pred_labels = logits.argmax(dim=1)  # 获取预测的类别
        acc = (pred_labels == labels).sum().item() / len(labels)
        train_loss.append(loss.item())
        train_accs.append(acc)
        

    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    writer.add_scalar('Train Loss', train_loss, epoch)
    writer.add_scalar('Train Accuracy', train_acc, epoch)

    print(f"{epoch + 1} epoch: loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ------------验证部分-----------
    model.eval()
    valid_loss = []
    valid_accs = []
    for batch in tqdm(valid_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.long().to(device)
        
        with torch.no_grad():
            logits = model(imgs)
        loss = loss_function(logits, labels)

        pred_labels = logits.argmax(dim=1)
        acc = (pred_labels == labels).sum().item() / len(labels)

        valid_loss.append(loss.item())
        valid_accs.append(acc)
        
      
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)
    writer.add_scalar('Valid Loss', valid_loss, epoch)
    writer.add_scalar('Valid Accuracy', valid_acc, epoch)

    #记录最佳参数
    if valid_acc > best_acc:
        print(f"{epoch + 1} epoch: loss = {valid_loss:.5f}, acc = {valid_acc:.5f}. Best acc on valid set.")
        print("save model")
        torch.save(model.state_dict(), "best_parameter.ckpt")
        best_acc = valid_acc
    else:
        print(f"{epoch + 1} epoch: loss = {valid_loss:.5f}, acc = {valid_acc:.5f}.")

writer.close()
print("done")
        

 14%|█▍        | 13/93 [00:00<00:00, 122.34it/s]

1 epoch: loss = 0.99935, acc = 0.70610


  2%|▏         | 7/326 [00:00<00:04, 68.81it/s]

1 epoch: loss = 0.38895, acc = 0.88575. Best acc on valid set.
save model


 14%|█▍        | 13/93 [00:00<00:00, 127.66it/s]

...

19 epoch: loss = 0.03697, acc = 0.99059. Best acc on valid set.
save model


 14%|█▍        | 13/93 [00:00<00:00, 128.40it/s]

20 epoch: loss = 0.03690, acc = 0.98869


100%|██████████| 93/93 [00:00<00:00, 126.16it/s]

20 epoch: loss = 0.04522, acc = 0.98723.
done


# 测试

In [14]:
test_loader = DataLoader(test_set, batch_size=batch_size,
                         shuffle=False, pin_memory=True)

model_best = CNN().to(device)
model_best.load_state_dict(torch.load("best_parameter.ckpt"))
model_best.eval()
test_accs = []
with torch.no_grad():
    for data, labels in tqdm(test_loader):
        data, labels = data.to(device), labels.to(device)
        logits = model_best(data)
        pred_labels = logits.argmax(dim=1)  # 获取预测的类别
        acc = (pred_labels == labels).sum().item() / len(labels)
        test_accs.append(acc)

    test_acc = sum(test_accs) / len(test_accs)
    print(f"testdata acc = {test_acc:.5f}")


  0%|          | 0/47 [00:00<?, ?it/s]

100%|██████████| 47/47 [00:00<00:00, 124.66it/s]

testdata acc = 0.99202


# 面试

In [15]:
test_data_set = ChineseDataset('./train_data/test', tfm=test_tfm)
test_loader = DataLoader(test_data_set, batch_size=batch_size,
                         shuffle=False, pin_memory=True)

model_best = CNN().to(device)
model_best.load_state_dict(torch.load("best_parameter.ckpt"))
model_best.eval()
test_accs = []
with torch.no_grad():
    for data, label in tqdm(test_loader):
        data, label = data.to(device), label.to(device)
        logits = model(data)
        pred_label = logits.argmax(dim=1)  # 获取预测的类别
        acc = (pred_label == label).sum().item() / len(label)
        test_accs.append(acc)

    test_acc = sum(test_accs) / len(test_accs)
    print(f"test acc = {test_acc:.5f}")


  0%|          | 0/180 [00:00<?, ?it/s]

100%|██████████| 180/180 [00:01<00:00, 168.80it/s]

test acc = 0.98368


In [1]:
test_data_set = ChineseDataset('./train_data/test', tfm=test_tfm)
test_loader = DataLoader(test_data_set, batch_size=batch_size,
                         shuffle=False, pin_memory=True)

model_best = CNN().to(device)
model_best.load_state_dict(torch.load("best_parameter.ckpt"))
model_best.eval()
test_accs = []
with torch.no_grad():
    for data, label in tqdm(test_loader):
        data, label = data.to(device), label.to(device)
        logits = model(data)
        pred_label = logits.argmax(dim=1)  # 获取预测的类别
        acc = (pred_label == label).sum().item() / len(label)
        test_accs.append(acc)

    test_acc = sum(test_accs) / len(test_accs)
    print(f"test acc = {test_acc:.5f}")


NameError: name 'ChineseDataset' is not defined